In [ ]:
import sys
import os

# Get the root directory by moving up one level from "notebooks/"
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add root directory to sys.path
sys.path.append(root_dir)

# Now you can import using the module path
from pipeline.utils.duckdb_wrapper import DuckDBWrapper


In [ ]:
# Initialize the DuckDBWrapper (in-memory DuckDB instance) You can connect directly to a DuckDB file by adding the path like con = DuckDBWrapper()
con = DuckDBWrapper()

In [ ]:

partitioned_table_names = [
    "nyc_threeoneone_requests",
    "mta_subway_origin_destination_2023",
    "mta_subway_origin_destination_2024",
    "mta_subway_hourly_ridership"
]

# Suppose your directory structure is something like:
repo_root = Path.cwd().resolve().parents[0]  # Adjust to locate the repo root
base_path = "data/opendata"

con.bulk_register_partitioned_data(
    repo_root=repo_root,
    base_path=base_path,
    table_names=partitioned_table_names,
    wildcard="year=*/month=*/*.parquet",
    show_tables=True
)


In [ ]:
query = f"""

SELECT 
count(*) as rows, 
MIN(created_date) as min,
MAX(created_date) as max
from 
nyc_threeoneone_requests
"""

result = con.run_query(query)

print(result)


In [ ]:
#If you want a better looking table, set show_results=True. I'd recomend capping the limit at about 50 rows
#T

query = f"""
SELECT 
 * from 
mta_subway_hourly_ridership limit 10
"""

result = con.run_query(query,show_results=True)


In [ ]:
# Show the tables registered
con.show_tables()


In [ ]:
# Show the schema of a specific table
con.show_schema("mta_subway_hourly_ridership")

In [ ]:
repo_root = Path.cwd().resolve().parents[0]  # Adjust to locate the repo root
base_path = repo_root / "data/exports"
file_name = "mta_subway_hourly_ridership_data_sample"
file_type= "csv"
# Export the query result to CSV
con.export(result, file_type=file_type, base_path=base_path, file_name=file_name)